# Web mapping lane-distance sketches

Small, self-contained examples for fake lane-segment workflows using the `geodist` Python API.

In [1]:
from geodist import (
    BoundingBox,
    Ellipsoid,
    LineString,
    Point,
    Point3D,
    geodesic_distance,
    geodesic_distance_3d,
    geodesic_distance_on_ellipsoid,
    geodesic_with_bearings_on_ellipsoid,
    hausdorff,
    hausdorff_clipped,
    hausdorff_directed,
)


## Quick lane-to-lane spacing (midpoints)

Measure the gap between two midpoint markers in a tile overlay.

In [2]:
lane_a_mid = Point(37.7749, -122.4194)
lane_b_mid = Point(37.7751, -122.4197)

midpoint_gap_m = geodesic_distance(lane_a_mid, lane_b_mid)
midpoint_gap_m

34.49361663433557

## Endpoint distance with approach bearings

Capture distance and bearings for a merge graphic.

In [3]:
origin = Point(47.6205, -122.3493)
destination = Point(47.6220, -122.3475)

result = geodesic_with_bearings_on_ellipsoid(origin, destination)
result

GeodesicResult(distance_m=214.75890772966508, initial_bearing_deg=39.052124089105504, final_bearing_deg=39.053453758783846)

## Surveyed vs mapped centerline (Hausdorff witness)

Flag the worst deviation between a surveyed trace and a styled centerline.

In [4]:
surveyed = LineString([(40.7128, -74.0060), (40.7135, -74.0055)])
mapped = LineString([(40.7127, -74.0061), (40.7136, -74.0054)])

witness = hausdorff_directed(surveyed.densify(), mapped.densify())
witness

HausdorffDirectedWitness(distance_m=13.952867651329369, origin_index=0, candidate_index=0)

## Clip comparison to an intersection box

Ignore faraway vertices by clipping both sets before comparison.

In [5]:
box = BoundingBox(40.7125, 40.7140, -74.0065, -74.0050)

witness = hausdorff_clipped(
    surveyed.densify(),
    mapped.densify(),
    box,
)
witness

HausdorffWitness(distance_m=44.256294995432654, a_to_b=HausdorffDirectedWitness(distance_m=13.952867651329369, origin_index=0, candidate_index=0), b_to_a=HausdorffDirectedWitness(distance_m=44.256294995432654, origin_index=1, candidate_index=0))

## Lane shoulder vs curb polyline offset

Compare offset polylines that approximate shoulder vs curb edges.

In [6]:
shoulder = LineString(
    [
        (34.0524, -118.2440),
        (34.0524, -118.2434),
        (34.0520, -118.2434),
    ]
)
curb = LineString(
    [
        (34.0523, -118.2439),
        (34.0523, -118.2435),
        (34.0521, -118.2435),
    ]
)

offset_m = hausdorff(
    shoulder.densify(max_segment_length_m=5.0),
    curb.densify(max_segment_length_m=5.0),
).distance_m
offset_m

14.440215825429576

## Altitude separation for stacked ramps

Check vertical clearance where stacked ramps overlap in XY.

In [7]:
upper = Point3D(39.7390, -104.9903, 18.0)
lower = Point3D(39.7390, -104.9903, 5.0)

vertical_gap_m = geodesic_distance_3d(upper, lower)
vertical_gap_m

13.00000000088197

## Densify a lane polyline before vector export

Pre-sample vertices for smoother rendering in vector tiles.

In [8]:
raw_lane = LineString(
    [
        (51.5007, -0.1246),
        (51.5015, -0.1230),
        (51.5020, -0.1215),
    ]
)

samples = raw_lane.densify(
    max_segment_length_m=5.0,
    max_segment_angle_deg=0.05,
)
len(samples)

54

## Distance on a custom ellipsoid

Use a grid-compliant ellipsoid for distance checks inside a local map projection.

In [9]:
grs80 = Ellipsoid(6378137.0, 6356752.314140)
p1, p2 = Point(52.5200, 13.4050), Point(52.5205, 13.4090)

dist_m = geodesic_distance_on_ellipsoid(p1, p2, grs80)
dist_m

277.15817366605376